In [37]:
import pandas as pd

df = pd.read_excel('./All.xlsx', sheet_name=0, parse_dates=['监测时间'], index_col='监测时间')
df.head()

,省份,城市,水温,pH,溶解氧,高锰酸钾,氨氮,总磷,总氮,电导率,浊度,人口数量,土地利用,植被覆盖,土壤,GDP_百万
监测时间,,,,,,,,,,,,,,,,
2022-01-01,山东省,济宁市,4.50700,8.18500,13.55600,5.44400,0.0430,0.11300,7.71300,2188.7000,45.57100,77.0,1.0,4216.0,11627.0,5.390217
2022-01-01,安徽省,六安市,6.17272,8.62618,14.47377,1.65111,0.0250,0.03741,0.80000,143.0002,2.33642,85.0,8.0,32767.0,11389.0,2.409009
2022-01-01,江苏省,扬州市,8.37000,8.14847,10.44229,3.73838,0.2900,0.11781,2.80989,583.0605,78.94016,1318.0,8.0,3769.0,11673.0,199.969620
2022-01-01,安徽省,宿州市,5.73000,8.10220,11.40000,4.31200,0.2029,0.04990,8.65550,1131.9900,5.12900,333.0,8.0,5858.0,11531.0,8.191306
2022-01-01,江苏省,徐州市,4.62000,7.56000,9.68000,0.25000,0.0250,0.01580,23.60400,952.2000,19.50000,788.0,1.0,3686.0,11435.0,8.904672


In [38]:
target_value = '淮安市'
# 使用条件筛选出包含目标值的所有行
filtered_df = df[df['城市'] == target_value]

# 对同一时间点的各列数据进行平均计算
average_by_time = filtered_df.resample('4H').mean()  

In [39]:
df = average_by_time.dropna()
X = df.drop(columns=['高锰酸钾']).values
y = df['高锰酸钾'].values.reshape(-1, 1)
X.shape, y.shape

((2136, 13), (2136, 1))

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1708, 13), (428, 13), (1708, 1), (428, 1))

In [41]:
from sklearn.preprocessing import StandardScaler

# 标准化
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# 注意：这里对训练集和测试集分别做标准化
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

y_train = y_train.ravel()
y_test = y_test.ravel()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1708, 13), (428, 13), (1708,), (428,))

In [42]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR

model = SVR(kernel='rbf')
model.fit(X_train, y_train)

train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

# 反标准化
train_pred = scaler_y.inverse_transform(train_pred.reshape(-1, 1)).ravel()
test_pred = scaler_y.inverse_transform(test_pred.reshape(-1, 1)).ravel()
print(test_pred)

print(f"训练集效果 MSE: {mean_squared_error(y_train, train_pred):.4f}, R²: {r2_score(y_train, train_pred):.4f}")
print(f"测试集效果 MSE: {mean_squared_error(y_test, test_pred):.4f}, R²: {r2_score(y_test, test_pred):.4f}")

[4.39389084 3.81007901 4.05432728 3.70282626 3.47087496 3.76795593
 5.52575702 3.22207796 3.73635773 4.1730158  3.97801108 4.48620572
 4.27007687 3.94589375 3.73201254 4.01059467 4.4899047  3.61298263
 4.28417799 3.67182967 4.79283161 3.93970199 4.05733986 3.36625737
 5.00316943 4.12329027 3.70031957 3.83213159 3.64856762 3.94979717
 3.30389918 3.9237316  4.07327279 4.78633003 2.96923379 3.93510582
 4.75086187 4.72223691 3.69617118 3.02761779 3.41528685 3.70488745
 5.62122553 4.59265497 4.88915625 4.19685077 4.45199936 3.16971303
 4.72543729 3.60171213 4.56177276 4.17904491 4.70267675 3.81850459
 4.54954321 5.51966992 4.21489092 4.54573027 4.6560346  4.52667618
 3.88079154 3.68961864 3.47129797 3.90640189 3.77260977 5.18533706
 4.72087734 3.68140243 4.58909159 4.67020539 4.98870271 4.31249048
 4.09193619 3.70092635 4.61261368 3.77300806 3.31347989 3.00242272
 4.06272094 5.17380578 4.69911373 4.49384905 4.80015951 3.73608274
 3.92803826 4.74461865 4.71390553 3.86367241 3.62330564 4.7173